# Neural Networks Assignment

## Overview
This assignment implements neural network components as follows:
- **Part 1**: Logic gates (AND, OR, NOR, NAND, XOR) using neurons with sigmoid activation.
- **Part 2**: Perceptron for AND and OR gates.
- **Part 3**: Feedforward neural network with sigmoid and softmax activations for single and batch inputs.
- **Part 4**: Debugging incorrect logic gate parameters to demonstrate understanding of errors.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Sigmoid function
def sigmoid(x):
    """Sigmoid function: 1 / (1 + e^-x)"""
    return 1.0 / (1.0 + np.exp(-x))

# Plot sigmoid function
vals = np.linspace(-10, 10, num=100, dtype=np.float32)
activation = sigmoid(vals)
fig = plt.figure(figsize=(12,6))
plt.plot(vals, activation)
plt.grid(True, which='both')
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.yticks()
plt.ylim([-0.5, 1.5])
plt.title('Sigmoid Activation Function')
plt.xlabel('Input')
plt.ylabel('Output')
plt.show()

## Part 1: Neurons as Logic Gates

We implement logic gates using a single neuron with sigmoid activation, where $z = w_1 x_1 + w_2 x_2 + b$ and output is $\sigma(z)$. Inputs $x_1, x_2 \in \{0, 1\}$, and weights/biases are chosen to produce outputs close to 0 or 1 per the truth tables.

In [ ]:
# Logic gate helper function
def logic_gate(w1, w2, b):
    """Create a logic gate with weights w1, w2 and bias b"""
    return lambda x1, x2: sigmoid(w1 * x1 + w2 * x2 + b)

# Test function for logic gates
def test(gate, gate_name):
    """Test logic gate and print truth table"""
    print(f"{gate_name} Gate:")
    for a, b in [(0, 0), (0, 1), (1, 0), (1, 1)]:
        print(f"{a}, {b}: {np.round(gate(a, b))}")

### OR Gate

<table>
<tr><th colspan="3">OR gate truth table</th></tr>
<tr><th colspan="2">Input</th><th>Output</th></tr>
<tr><td>0</td><td>0</td><td>0</td></tr>
<tr><td>0</td><td>1</td><td>1</td></tr>
<tr><td>1</td><td>0</td><td>1</td></tr>
<tr><td>1</td><td>1</td><td>1</td></tr>
</table>

Parameters: $w_1=20, w_2=20, b=-10$ ensure $z$ is negative for (0,0) and positive otherwise.

In [ ]:
or_gate = logic_gate(20, 20, -10)
test(or_gate, "OR")

### AND Gate

<table>
<tr><th colspan="3">AND gate truth table</th></tr>
<tr><th colspan="2">Input</th><th>Output</th></tr>
<tr><td>0</td><td>0</td><td>0</td></tr>
<tr><td>0</td><td>1</td><td>0</td></tr>
<tr><td>1</td><td>0</td><td>0</td></tr>
<tr><td>1</td><td>1</td><td>1</td></tr>
</table>

Parameters: $w_1=15, w_2=15, b=-20$ ensure $z$ is positive only for (1,1).

In [ ]:
and_gate = logic_gate(15, 15, -20)
test(and_gate, "AND")

### NOR Gate

<table>
<tr><th colspan="3">NOR gate truth table</th></tr>
<tr><th colspan="2">Input</th><th>Output</th></tr>
<tr><td>0</td><td>0</td><td>1</td></tr>
<tr><td>0</td><td>1</td><td>0</td></tr>
<tr><td>1</td><td>0</td><td>0</td></tr>
<tr><td>1</td><td>1</td><td>0</td></tr>
</table>

Parameters: $w_1=-20, w_2=-20, b=10$ ensure $z$ is positive for (0,0) and negative otherwise.

In [ ]:
nor_gate = logic_gate(-20, -20, 10)
test(nor_gate, "NOR")

### NAND Gate

<table>
<tr><th colspan="3">NAND gate truth table</th></tr>
<tr><th colspan="2">Input</th><th>Output</th></tr>
<tr><td>0</td><td>0</td><td>1</td></tr>
<tr><td>0</td><td>1</td><td>1</td></tr>
<tr><td>1</td><td>0</td><td>1</td></tr>
<tr><td>1</td><td>1</td><td>0</td></tr>
</table>

Parameters: $w_1=-15, w_2=-15, b=20$ ensure $z$ is negative only for (1,1).

In [ ]:
nand_gate = logic_gate(-15, -15, 20)
test(nand_gate, "NAND")

### XOR Gate

<table>
<tr><th colspan="3">XOR gate truth table</th></tr>
<tr><th colspan="2">Input</th><th>Output</th></tr>
<tr><td>0</td><td>0</td><td>0</td></tr>
<tr><td>0</td><td>1</td><td>1</td></tr>
<tr><td>1</td><td>0</td><td>1</td></tr>
<tr><td>1</td><td>1</td><td>0</td></tr>
</table>

XOR is implemented as $AND(OR(x_1, x_2), NAND(x_1, x_2))$, as a single neuron cannot model XOR.

In [ ]:
def xor_gate(a, b):
    """XOR gate using OR, NAND, and AND gates"""
    c = or_gate(a, b)
    d = nand_gate(a, b)
    return and_gate(c, d)

test(xor_gate, "XOR")

## Part 2: Perceptron for AND and OR Gates

We implement a perceptron with a step function activation to model AND and OR gates, which are linearly separable.

In [ ]:
# Perceptron class
class Perceptron:
    def __init__(self, weights, bias):
        self.weights = np.array(weights)
        self.bias = bias

    def step_function(self, x):
        return 1 if x >= 0 else 0

    def predict(self, inputs):
        weighted_sum = np.dot(inputs, self.weights) + self.bias
        return self.step_function(weighted_sum)

# Test perceptron
def test_perceptron(perceptron, gate_name):
    print(f"{gate_name} Perceptron:")
    inputs = [(0, 0), (0, 1), (1, 0), (1, 1)]
    for x1, x2 in inputs:
        print(f"{x1}, {x2}: {perceptron.predict([x1, x2])}")

# AND Perceptron: w1=1, w2=1, b=-1.5
and_perceptron = Perceptron([1, 1], -1.5)
test_perceptron(and_perceptron, "AND")

# OR Perceptron: w1=1, w2=1, b=-0.5
or_perceptron = Perceptron([1, 1], -0.5)
test_perceptron(or_perceptron, "OR")

## Part 3: Feedforward Neural Network

We compute the forward pass of a three-layer neural network with sigmoid activations for hidden layers and softmax for the output layer. We provide functions for single and batch inputs.

In [ ]:
# Weight matrices
W_1 = np.array([[2, -1, 1, 4], [-1, 2, -3, 1], [3, -2, -1, 5]])
W_2 = np.array([[3, 1, -2, 1], [-2, 4, 1, -4], [-1, -3, 2, -5], [3, 1, 1, 1]])
W_3 = np.array([[-1, 3, -2], [1, -1, -3], [3, -2, 2], [1, 2, 1]])

# Input data
x_in = np.array([0.5, 0.8, 0.2])
x_mat_in = np.array([
    [0.5, 0.8, 0.2],
    [0.1, 0.9, 0.6],
    [0.2, 0.2, 0.3],
    [0.6, 0.1, 0.9],
    [0.5, 0.5, 0.4],
    [0.9, 0.1, 0.9],
    [0.1, 0.8, 0.7]
])

# Softmax functions
def soft_max_vec(vec):
    return np.exp(vec) / np.sum(np.exp(vec))

def soft_max_mat(mat):
    return np.exp(mat) / np.sum(np.exp(mat), axis=1).reshape(-1, 1)

# Compute layer inputs and outputs for x_in
z1 = x_in @ W_1
a1 = sigmoid(z1)
z2 = a1 @ W_2
a2 = sigmoid(z2)
z3 = a2 @ W_3
output = soft_max_vec(z3)

print("Layer 1 input (z1):", z1)
print("Layer 1 output (a1):", a1)
print("Layer 2 input (z2):", z2)
print("Layer 2 output (a2):", a2)
print("Layer 3 input (z3):", z3)
print("Network output (softmax):", output)

# Functions for neural network forward pass
def nn_forward_single(x):
    """Forward pass for single input"""
    a1 = sigmoid(x @ W_1)
    a2 = sigmoid(a1 @ W_2)
    out = soft_max_vec(a2 @ W_3)
    return out

def nn_forward_batch(X):
    """Forward pass for batch inputs"""
    a1 = sigmoid(X @ W_1)
    a2 = sigmoid(a1 @ W_2)
    out = soft_max_mat(a2 @ W_3)
    return out

# Test functions
print("\nSingle input output:", nn_forward_single(x_in))
print("Batch input output:\n", nn_forward_batch(x_mat_in))

## Part 4: Debugging Incorrect Logic Gate Parameters

We analyze the impact of incorrect NOR and NAND parameters (`w1=0, w2=0, b=0`) observed in some submissions, which caused XOR to fail.

In [ ]:
# Test incorrect NOR and NAND gates
incorrect_nor_gate = logic_gate(0, 0, 0)
incorrect_nand_gate = logic_gate(0, 0, 0)

print("Incorrect NOR Gate (w1=0, w2=0, b=0):")
test(incorrect_nor_gate, "NOR")

print("\nIncorrect NAND Gate (w1=0, w2=0, b=0):")
test(incorrect_nand_gate, "NAND")

# Test XOR with incorrect NOR
def incorrect_xor_gate_nor(a, b):
    c = or_gate(a, b)
    d = incorrect_nor_gate(a, b)
    return and_gate(c, d)

print("\nXOR with Incorrect NOR:")
test(incorrect_xor_gate_nor, "XOR")

# Test XOR with incorrect NAND
def incorrect_xor_gate_nand(a, b):
    c = or_gate(a, b)
    d = incorrect_nand_gate(a, b)
    return and_gate(c, d)

print("\nXOR with Incorrect NAND:")
test(incorrect_xor_gate_nand, "XOR")

### Debugging Analysis

- **Incorrect Parameters** (`w1=0, w2=0, b=0`):
  - For all inputs, $z = 0$, so $\sigma(z) = 0.5$, rounded to `0.0` in tests.
  - NOR should output `(0,0:1), (0,1:0), (1,0:0), (1,1:0)`; instead, all outputs are `0`.
  - NAND should output `(0,0:1), (0,1:1), (1,0:1), (1,1:0)`; instead, all outputs are `0`.

- **Impact on XOR**:
  - XOR = $AND(OR(a,b), NAND(a,b))$.
  - With incorrect NAND, `NAND(a,b)=0` for all inputs, so `AND(c,0)=0`, causing XOR to output `0` for all inputs.
  - With incorrect NOR, similar issues arise, as `NOR(a,b)=0` disrupts the AND gate logic.
  - This explains the incorrect XOR output `(1,1:1)` in some submissions, as the intermediate gates fail.

- **Correct Parameters**:
  - NOR: `w1=-20, w2=-20, b=10` produces $z=10$ for (0,0) and negative otherwise.
  - NAND: `w1=-15, w2=-15, b=20` produces $z=20$ for non-(1,1) inputs and $z=-10$ for (1,1).
  - These ensure correct XOR: `(0,0:0), (0,1:1), (1,0:1), (1,1:0)`.